In [23]:
import pandas as pd
import plotly.express as px

# Load the CSV
df = pd.read_csv('data/isolation_and_bottles.csv')

# Convert last_changed to datetime
df['last_changed'] = pd.to_datetime(df['last_changed'])

# Map entity IDs to nicer names
entity_mapping = {
    'sensor.ds18b20_sensor_288ba6': 'Electronics',
    'sensor.ds18b20_sensor_28d206': 'Outside',
    'sensor.ds18b20_sensor_2805e1': 'Gray bin',
    'sensor.ds18b20_sensor_2855b9': 'Wooden box (outside wool)',
}

df['entity_name'] = df['entity_id'].map(entity_mapping)

# Plot using the nicer names
fig = px.line(
    df,
    x='last_changed',
    y='state',
    color='entity_name',
    title='WiggleBin with wool insulation and 2 bottles of water'
)

# Hide Electronics and Outside by default
hidden_entities = ['Electronics', 'Outside']
for trace in fig.data:
    if trace.name in hidden_entities:
        trace.visible = 'legendonly'

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    legend_title='Sensor'
)

fig.show()